In [1]:
%run /AAF/AAF_TLD/Conf/Configuration

In [2]:
fact_final1 = sql("select  CAI_CODE,CAI_MARKET_FDM_SK,level0_desc as customer,level3_desc,DT_YEARMONTH,COUNTRY_CODE,SUM(SALES_QTY) as SALES_QTY,SUM(SELL_IN_EURO) as SELL_IN_EURO,SELL_IN_LOC,LB_CODE,mini,max, FLAG_ANOMALY,FLAG_CORRIDOR,gap,MFS_IDR,MARKET_CODE from fact_final a LEFT JOIN custpricing_df d ON a.cust_pricing_fdm_sk = d.cust_pricing_fdm_sk LEFT JOIN cust_lvl0 cl ON d.level0_code = cl.level0_code  LEFT JOIN cust_hiedf ch ON ch.level0_code = cl.level0_code  LEFT JOIN cust_lvl3 c3 ON ch.level3_code = c3.level3_code  group by CAI_CODE,CAI_MARKET_FDM_SK,level0_desc,DT_YEARMONTH,COUNTRY_CODE,SALES_QTY,SELL_IN_LOC,LB_CODE,mini,max,MFS_IDR,MARKET_CODE ,FLAG_ANOMALY,FLAG_CORRIDOR,gap,level3_desc")
fact_final1.registerTempTable("fact_final1")

In [3]:
%python
import pandas as pd
country_fact = sql("""SELECT DISTINCT b.country_michelin_en_desc       AS Country, 
                                      a.dt_yearmonth                   AS Month, 
                                      Customer                      AS Customer, 
                                      level3_desc                     As Cust_level3_desc,
                                      a.cai_code                       AS CAI, 
                                      a.sales_qty                      AS Quantity, 
                                      case when  FLAG_ANOMALY IN('B','A') THEN a.sales_qty else 0 end  AS Quantity_in_anomaly, 
                                      a.sell_in_euro                   AS Sell_In_Net_Sales, 
                                      ( a.sell_in_euro / a.sales_qty ) AS Sell_In_Net_Sales_per_unit, 
                                      mini                  AS Min_Corridor_Price, 
                                      max                  AS Max_Corridor_Price, 
                                     gap as  Average_GAP_vs_Corridor, 
                                      CASE  when  FLAG_ANOMALY IN('B','A') THEN sell_in_euro else 0 end  AS Sell_In_Net_Sales_in_anomaly,
                                      cm.etrma_seat_dia_code              AS seat, 
                                      season_s_w_desc                  AS season, 
                                      rc.region_code                   AS REGION,
                                      a.LB_CODE,
                                      SELL_IN_LOC,
                                      cd.CAI_DESC,
                                      bl.BRAND_DESC as brand,
                                      concat(cd.ETRMA_BOUD_CODE,'/',cd.ETRMA_RHS_CODE,' ',cd.ETRMA_STRU_CODE,cd.ETRMA_SEAT_DIA_CODE,' ',cd.ETRMA_CH_I_CODE,cd.ETRMA_VIT_CODE) as DIMBOX,                                                       concat(cd.ETRMA_BOUD_CODE,'/',cd.ETRMA_RHS_CODE,' ',cd.ETRMA_STRU_CODE,cd.ETRMA_SEAT_DIA_CODE) as GEOBOX,
                                      MFS_IDR,MARKET_CODE,p5_desc,FLAG_ANOMALY
                  FROM   fact_final1 a 
                         LEFT JOIN country_df b 
                                ON a.country_code = b.country_michelin_code 
                         LEFT JOIN region_countrydf rc 
                                ON b.country_michelin_code = rc.country_michelin_code 
                         LEFT JOIN regiondf r 
                                ON rc.region_code = r.region_code 
                        
                         LEFT JOIN cai_mark cm 
                                ON a.cai_market_fdm_sk = cm.cai_market_fdm_sk 
                         left join prod_heir 
                                 on SEGMENT_ASR_CODE =P1_CODE
                         LEFT JOIN cai_seas se 
                                ON cm.season_code = se.a06 
                                LEFT JOIN cdi_desc cd 
                                 on cast(a.cai_code as int) = cast(cd.cai_code as int)
                         left join brand_load bl
                               on cast (cd.BRAND_CODE as int) = cast(bl.BRAND_CODE as int)
              """)

country_fact.registerTempTable("country_fact")

In [4]:
#final output of data preparation
finaldata = sql("""SELECT distinct country, 
                          month, 
                          customer,
                          Cust_level3_desc,
                          cai, 
                          quantity, 
                          sell_in_net_sales, 
                          sell_in_net_sales_per_unit, 
                          min_corridor_price, 
                          max_corridor_price, 
                          average_gap_vs_corridor, 
                          quantity_in_anomaly, 
                          ( average_gap_vs_corridor * quantity_in_anomaly ) AS Economic_impact, 
                          CASE 
                            WHEN quantity_in_anomaly IS NULL THEN 0 
                            ELSE quantity_in_anomaly 
                          END                                               AS  Quantity_in_anomaly_checked, 
                          sell_in_net_sales_in_anomaly, 
                          CASE 
                            WHEN ( average_gap_vs_corridor * quantity_in_anomaly ) IS NULL 
                          THEN 0 
                            ELSE ( average_gap_vs_corridor * quantity_in_anomaly ) 
                          END                                               AS  Economic_impact_checked, 
                          CASE 
                            WHEN average_gap_vs_corridor != 0 THEN ( 
                            average_gap_vs_corridor / min_corridor_price ) 
                            ELSE 0 
                          END                                               AS  Averagegap_Mincorr_Price_ratio, 
                           
                          seat                                              AS Seat, 
                          season                                            AS Season_A_S_W, 
                          CASE 
                            WHEN country = 'THAILAND' THEN 'E2A' 
                            WHEN country = 'INDIA' THEN 'INDIA' 
                            WHEN country = 'CHINA' THEN 'CHN' 
                            WHEN region = 'AIM' THEN 'AFMO' 
                            ELSE region 
                          END                                               AS region ,
                          SELL_IN_LOC,
                          CAI_DESC,
                          brand,
                          DIMBOX,
                          GEOBOX,
                          MFS_IDR,MARKET_CODE,p5_desc,FLAG_ANOMALY
              FROM   country_fact """)

finaldata.registerTempTable("finaldata")



In [5]:
%python

import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from statistics import mean 
pd.options.display.max_rows = 2
pd.options.display.max_columns = 200

#final = sql("select a.*, cast(substr(a.Month,0,4) as int) as curr_year from finaldata a where Min_Corridor_Price !=0 and Max_Corridor_Price !=0 ")
#datatto.registerTempTable("datatto")
#final = sql("select a.*, cast(substr(a.Month,0,4) as int) as curr_year from fact_filt  a where Min_Corridor_Price !=0 and Max_Corridor_Price !=0 ")
final = sql("select a.*, cast(substr(a.Month,0,4) as int) as curr_year from finaldata  a  ")
#final.registerTempTable("d2")
#final.write.saveAsTable("df_All1802")




In [6]:
%python
%paste 

TempFilePath = _40_project + "/MAD/AGB/ParellelFlow_TLD/TMP/final"
final.coalesce(1).write\
       .mode("overwrite")\
       .format("com.databricks.spark.csv")\
       .option("header", "true")\
       .save(TempFilePath)\
######## now read file from temp location write it to new location with new name and delete temp directory
readPath = _40_project + "/AAF/Alerting_Flow_TLD/TMP/final"
writePath =_40_project + "/AAF/Alerting_Flow_TLD/TMP
file_list = dbutils.fs.ls(readPath) #### List out all files in temp directory
for i in file_list:
  file_path = i[0]
  file_name = i[1]
 #file_name
fname = "final.csv"
for i in file_list:
  if i[1].startswith("part-00000"):read_name = i[1] #### find your temp file name
# # # #####Move it outside to the new_dir folder and rename
dbutils.fs.mv(readPath+"/"+read_name, writePath+"/"+fname)
# # # #Remove the empty folder
dbutils.fs.rm(readPath , recurse= True) 

In [7]:
df_All1802 = spark.read.format("csv").options(header='true', delimiter = ',').load(_40_project + "/AAF/Alerting_Flow_TLD/TMP/final.csv")
df_All1802.registerTempTable("df_All1802")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-195397896629602> in <module> 
 ----> 1 df_All1802 = spark . read . format ( "csv" ) . options ( header = 'true' , delimiter = ',' ) . load ( _40_project + "/AAF/Alerting_Flow_TLD/TMP/final.csv" ) 
 2 df_All1802 . registerTempTable ( "df_All1802" ) 

 NameError : name '_40_project' is not defined

In [8]:

data_sql = sql("select * from df_All1802 where curr_year  >= '2019'")
data_sql.registerTempTable("data_inter1")



In [9]:
data_sql1 = sql("""select distinct data_inter1.*, 
                           concat(year(add_months(TO_DATE(CONCAT(Month,'01'),'yyyyMMdd'),-12)),LPAD((cast(month(add_months(TO_DATE(CONCAT(Month,'01'),'yyyyMMdd'),-12))as varchar(2))),2,'0')) as start ,
                           concat(year(add_months(TO_DATE(CONCAT(Month,'01'),'yyyyMMdd'),-1)),LPAD((cast(month(add_months(TO_DATE(CONCAT(Month,'01'),'yyyyMMdd'),-1))as varchar(2))),2,'0')) as end ,
                           TO_DATE(CONCAT(Month,'01'),'yyyyMMdd')as sales_date 
                    from data_inter1""")
data_sql1.registerTempTable("data_inter2")


In [10]:
rc = sql("select  a.COUNTRY_CODE,COUNTRY_MICHELIN_EN_DESC,REGION_CODE  From country_df a join region_countrydf r on a.COUNTRY_CODE = r.COUNTRY_CODE  ")
rc.registerTempTable("rc")

dfi1 = sql("select a.*,PAYS_LIB as country  From dfi a left join rpc on a.rpc_c = rpc.rpc_c")
dfi1.registerTempTable("dfi1")

dfi2 = sql("select a.*,REGION_CODE as Region from dfi1 a left join rc on UPPER(a.country) = UPPER(rc.COUNTRY_MICHELIN_EN_DESC) ")
dfi2.registerTempTable("dfi2")

datefile = dfi2.toPandas()

In [11]:
%python
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from statistics import mean 
pd.options.display.max_rows = 2
pd.options.display.max_columns = 200

%python
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from statistics import mean 
pd.options.display.max_rows = 2
pd.options.display.max_columns = 200


datefile['CAI_C'].replace(to_replace=np.nan, value=0,inplace=True)
datefile['CAI_C'] = datefile['CAI_C'].astype(int)



datefile['PREV_FIN_DT'].replace(to_replace=np.nan, value='12/31/9999',inplace=True)
datefile['PREV_FIN_DT'] = datefile['PREV_FIN_DT'].apply(lambda x: (datetime.datetime.strptime(str(x), '%m/%d/%Y').date()).strftime('%Y-%m-%d'))


datefile['PREV_DEB_DT'].replace(to_replace=np.nan, value='12/31/9999',inplace=True)
datefile['PREV_DEB_DT'] = datefile['PREV_DEB_DT'].apply(lambda x: (datetime.datetime.strptime(str(x), '%m/%d/%Y').date()).strftime('%Y-%m-%d'))
#datefile = datefile.rename(columns={"SEAT": "Seat","SEASON":"season_a_s_w"})


#display(datefile)



#display(datefile)


UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).

In [12]:

date_df =sqlContext.createDataFrame(pd.DataFrame(datefile))
date_df.registerTempTable("date_df")



In [13]:
date_df1 = sql("select a.*,PREV_DEB_DT as livm_region,PREV_FIN_DT as supp_region  From date_df a")
date_df1.registerTempTable("date_df1")

In [14]:
data_sql2 = sql("""SELECT distinct country, 
                                    month, 
                                    customer,
                                    Cust_level3_desc,
                                    cai, 
                                    quantity, 
                                    sell_in_net_sales, 
                                    sell_in_net_sales_per_unit, 
                                    min_corridor_price, 
                                    max_corridor_price, 
                                    average_gap_vs_corridor, 
                                    quantity_in_anomaly, 
                                    economic_impact, 
                                    quantity_in_anomaly_checked, 
                                    sell_in_net_sales_in_anomaly, 
                                    economic_impact_checked, 
                                    averagegap_mincorr_price_ratio, 
                                    dseat as seat, 
                                    Season_A_S_W as season_a_s_w, 
                                    curr_year, 
                                    start, 
                                    end, 
                                    sales_date,
                                    supp_region,
                                    livm_region,
                                    region, 
                                    SELL_IN_LOC,
                                    CASE 
                                      WHEN cast(Months_between(a.sales_date, livm_region) as int) BETWEEN 0 AND 
                                           12 THEN 'YES' 
                                      ELSE 'NO' 
                                    END AS phase_in, 
                                    CASE 
                                      WHEN cast(Months_between(supp_region, a.sales_date) as int) <= 3 THEN 'YES' 
                                      ELSE 'NO' 
                                    END AS phase_out, 
                                    
                                    CAI_DESC,
                                     brand,
                                     DIMBOX,
                                     GEOBOX,
                                     MFS_IDR,MARKET_CODE,p5_desc,FLAG_ANOMALY
                  FROM   (SELECT distinct di.*, 
                                 d.cai_c    AS caii, 
                                 d.region AS rg, 
                                 d.livm_region, 
                                 d.supp_region, 
                                 
                                 d.SEAT_DIA_VAL as dseat
                                 
                                  
                          FROM   data_inter2 di 
                                 left join date_df1 d 
                                        ON Cast(di.cai AS INT) = Cast(d.cai_c AS INT) 
                                           and UPPER(di.Region) = UPPER(d.Region)
                                           AND CAST(MARKET_CODE AS INT) = CAST(MRC_C AS INT)
                                                )A""")
data_sql2.registerTempTable("data_inter3")


In [15]:

data_sql3 = sql(""" SELECT DISTINCT a.*, 
                           CASE 
                             WHEN ( Cast(quantity_in_anomaly_checked AS INT) <= 0 ) 
                                   OR ( Cast(sell_in_net_sales_in_anomaly AS INT) < 0 ) 
                                   OR ( Cast(economic_impact_checked AS INT) >= 0 ) 
                                   OR ( UPPER(phase_in) = 'YES' ) 
                                   OR ( UPPER(phase_out) = 'YES' ) THEN 'Yes' 
                             ELSE 'No' 
                           END                            AS filter, 
                           Lpad(Month(sales_date), 2, '0')AS curr_month 
                    FROM   data_inter3 a """)
data_sql3.registerTempTable("data_inter4")

In [16]:

%python
%paste 

TempFilePath = _40_project + "/AAF/Alerting_Flow_TLD/TMP/data_sql51"
data_sql3.coalesce(1).write\
       .mode("overwrite")\
       .format("com.databricks.spark.csv")\
       .option("header", "true")\
       .save(TempFilePath)\
######## now read file from temp location write it to new location with new name and delete temp directory
readPath = _40_project + "/AAF/Alerting_Flow_TLD/TMP/data_sql51"
writePath =_40_project + "/AAF/Alerting_Flow_TLD/TMP/"
file_list = dbutils.fs.ls(readPath) #### List out all files in temp directory
for i in file_list:
  file_path = i[0]
  file_name = i[1]
 #file_name
fname = "data_sql51.csv"
for i in file_list:
  if i[1].startswith("part-00000"):read_name = i[1] #### find your temp file name
# # # #####Move it outside to the new_dir folder and rename
dbutils.fs.mv(readPath+"/"+read_name, writePath+"/"+fname)
# # # #Remove the empty folder
dbutils.fs.rm(readPath , recurse= True) 

In [17]:

data_all1802 = spark.read.format("csv").options(header='true', delimiter = ',').load(_40_project + "/AAF/Alerting_Flow_TLD/TMP/data_sql51.csv")
data_all1802.registerTempTable("data_all1802")

In [18]:
#2019
data_final = sql("select * from data_all1802 where filter == 'No'")
data_final.registerTempTable("data")

#All
df_all = sql("select * from  df_All1802")
df_all.registerTempTable("df")

In [19]:
data_sql6a = sql("""SELECT distinct da.*, 
                          
                          
                          CASE 
                            WHEN filter = 'No' THEN (quantity/country_cust_sku_year_sales_mean ) 
                            ELSE 0 
                          END AS Sku_sales_gap_year ,
                          (Cast(curr_year AS INT)-1) AS prev_year
                  FROM   data_all1802 da 
                         LEFT JOIN (SELECT df.country, 
                                           df.customer, 
                                           df.cai, 
                                           data.month, 
                                           Sum(df.quantity)      AS country_cust_sku_year_sales, 
                                           Avg(df.quantity) AS 
                                           country_cust_sku_year_sales_mean 
                                    FROM   df 
                                           INNER JOIN data 
                                                   ON data.customer = df.customer 
                                                      AND data.cai = df.cai 
                                                      AND df.country = data.country 
                                    WHERE  ( df.month BETWEEN data.start AND data.end ) and cast(df.quantity as int) != 0 
                                    GROUP  BY df.customer, 
                                              df.cai, 
                                              df.country, 
                                              data.month
                                       )ccs 
                                ON da.country = ccs.country 
                                   AND da.customer = ccs.customer 
                                   AND da.cai = ccs.cai 
                                   AND da.month = ccs.month 
                        
                         LEFT JOIN  (SELECT DISTINCT 
                                                                 country, 
                                                                 customer, 
                                                                 month, 
                                                                 Sum(quantity) AS 
                                                                 country_cust_month_sales 
                                                 FROM   df 
                                                 GROUP  BY df.country, 
                                                           df.customer, 
                                                           df.month
                                          
                                        )ccm 
                                ON 
                                    da.country = ccm.country 
                                   AND da.customer = ccm.customer 
                                   AND da.month = ccm.month""")

data_sql6a.registerTempTable("data_inter6a")
#display(sql("select * from data_inter6a"))

In [20]:
%python
      data_sql7 = sql("""SELECT distinct  a.country,month,a.customer,Cust_level3_desc,cai,quantity,sell_in_net_sales,sell_in_net_sales_per_unit,min_corridor_price,max_corridor_price,average_gap_vs_corridor,quantity_in_anomaly,SELL_IN_LOC,economic_impact,quantity_in_anomaly_checked,sell_in_net_sales_in_anomaly,economic_impact_checked,averagegap_mincorr_price_ratio,seat,season_a_s_w,a.curr_year,start,end,sales_date,supp_region,livm_region,region,phase_in,phase_out,filter,curr_month,COALESCE(Sku_sales_gap_year,0) as Sku_sales_gap_year,COALESCE(prev_year,0) as prev_year,
                         COALESCE( CASE 
                            WHEN filter = 'No' THEN prev_year_sales_df 
                            ELSE 0 
                          END ,0)AS previous_year_sales,CAI_DESC,
                              brand,
                              DIMBOX,
                              GEOBOX,
                              MFS_IDR,MARKET_CODE,p5_desc,FLAG_ANOMALY
                    FROM   data_inter6a a 
                           LEFT JOIN (SELECT DISTINCT df.country, 
                                                      df.customer, 
                                                      da.curr_year, 
                                                      Sum(quantity)AS prev_year_sales_df 
                                      FROM   (SELECT DISTINCT customer, 
                                                              country, 
                                                              curr_year, 
                                                              prev_year 
                                              FROM   data_inter6a 
                                              WHERE  filter = 'No') da 
                                             INNER JOIN df 
                                                     ON df.country = da.country 
                                                        AND df.customer = da.customer 
                                                        AND df.curr_year = da.prev_year 
                                      GROUP  BY df.country, 
                                                df.customer, 
                                                da.curr_year)pys 
                                  ON a.customer = pys.customer 
                                     AND a.country = pys.country 
                                     AND a.curr_year = pys.curr_year 
                                     AND a.curr_year = pys.curr_year
                            """)

data_sql7.registerTempTable("data_inter7_output")
#data_sql7.write.saveAsTable("data_inter7_All")

In [21]:

data_inter8 = sql("""SELECT  distinct  country,month,customer,Cust_level3_desc,cai,COALESCE(quantity,0) as quantity,COALESCE(sell_in_net_sales,0) as sell_in_net_sales,COALESCE(sell_in_net_sales_per_unit,0) as sell_in_net_sales_per_unit,min_corridor_price,max_corridor_price,COALESCE(average_gap_vs_corridor,0) as average_gap_vs_corridor,COALESCE(quantity_in_anomaly,0) as quantity_in_anomaly,SELL_IN_LOC,COALESCE(economic_impact,0) as economic_impact,quantity_in_anomaly_checked,sell_in_net_sales_in_anomaly,economic_impact_checked,averagegap_mincorr_price_ratio,COALESCE(cast(seat as double),0) as seat,season_a_s_w,curr_year,start,end,sales_date,supp_region,livm_region,region,phase_in,phase_out,filter,curr_month,COALESCE(Sku_sales_gap_year,0) as Sku_sales_gap_year ,COALESCE(prev_year,0) as prev_year,COALESCE(previous_year_sales,0) as previous_year_sales,
                                      CASE
                                          WHEN UPPER(filter) ='NO'
                                               AND previous_year_sales =0 THEN 'Yes'
                                          ELSE 'No'
                                      END AS new_sku_flag,CAI_DESC,
                              brand,
                              DIMBOX,
                              GEOBOX,
                              MFS_IDR,MARKET_CODE,p5_desc,FLAG_ANOMALY
                      FROM data_inter7_output da""")
data_inter8.registerTempTable("data_inter8")


In [22]:
data_inter9 = sql("""SELECT distinct da.*,
                CASE
                    WHEN UPPER(filter) = 'NO'
                         AND economic_impact_checked >= -10000 THEN 1
                    ELSE 2
                END AS economic_impact_risk_inner,
                CASE
                    WHEN UPPER(filter) = 'NO'
                         AND quantity_in_anomaly_checked < 50 THEN 1
                    ELSE 2
                END AS quantity_risk_inner,
                CASE
                    WHEN UPPER(filter) = 'NO'
                         AND averagegap_mincorr_price_ratio >= -0.05 THEN 1
                    ELSE 2
                END AS average_gap_risk_inner,
                
                CASE
                    WHEN UPPER(filter) = 'NO'
                         AND Sku_sales_gap_year <= 1.2 THEN 1
                    ELSE 2
                END AS average_Sku_sales_gap_year_inner FROM data_inter8 da""")

data_inter9.registerTempTable("data_inter9")

In [23]:

data_inter10 = sql("""SELECT distinct da.*,
                CASE
                    WHEN UPPER(filter) = 'NO'
                         AND economic_impact_checked <= -25000
                         AND economic_impact_risk_inner = 2 THEN 3
                    ELSE economic_impact_risk_inner
                END AS economic_impact_risk,
                CASE
                    WHEN UPPER(filter) = 'NO'
                         AND quantity_in_anomaly_checked >= 300
                         AND quantity_risk_inner = 2 THEN 3
                    ELSE quantity_risk_inner
                END AS quantity_risk,
                CASE
                    WHEN UPPER(filter) = 'NO'
                         AND averagegap_mincorr_price_ratio <= -0.12
                         AND average_gap_risk_inner = 2 THEN 3
                    ELSE average_gap_risk_inner
                END AS average_gap_risk,
                
                
                CASE
                    WHEN UPPER(filter) = 'NO'
                         AND Sku_sales_gap_year > 1.5
                         AND average_Sku_sales_gap_year_inner =2 THEN 3
                    ELSE average_Sku_sales_gap_year_inner
                END AS average_Sku_sales_gap_year
                
FROM data_inter9 da""")
data_inter10.registerTempTable("data_inter10")



In [24]:


data_inter11= sql("""SELECT distinct da.*,
                            COALESCE (CASE
                                 WHEN UPPER(filter) = 'NO'
                                      AND UPPER(new_sku_flag)='NO' THEN (economic_impact_risk+quantity_risk+average_gap_risk+average_Sku_sales_gap_year)
                                 ELSE 0
                             END ,0) AS total_risk
                            
                            
                      FROM data_inter10 da""")
data_inter11.registerTempTable("data_inter11")

In [25]:

data_inter12= sql("""SELECT  da.*,
                              CASE
                                  WHEN total_risk >=9 THEN 'High'
                                  ELSE 'Medium'
                              END AS Risk_inner1
                              
                              
              FROM data_inter11 da""")
data_inter12.registerTempTable("data_inter12")

In [26]:

data_inter13 = sql("""SELECT distinct da.*,
                                      CASE
                                          WHEN total_risk <= 6
                                               AND Risk_inner1 = 'Medium' THEN 'Low'
                                          ELSE Risk_inner1
                                      END AS Risk_inner
                                      
                                      
                      FROM data_inter12 da""")
data_inter13.registerTempTable("data_inter13")

In [27]:
data_inter14 = sql("""SELECT distinct da.*,
                CASE
                    WHEN customer = 'PARALLEL EXPORT' THEN 'Low'
                    ELSE Risk_inner
                END AS Risk
FROM data_inter13 da""")
data_inter14.registerTempTable("data_inter14")

In [28]:
data_inter18 =sql("""SELECT distinct  country,month,customer,Cust_level3_desc,cai,quantity,sell_in_net_sales,sell_in_net_sales_per_unit,min_corridor_price,max_corridor_price,average_gap_vs_corridor,quantity_in_anomaly,sell_in_loc,economic_impact,quantity_in_anomaly_checked,sell_in_net_sales_in_anomaly,economic_impact_checked,averagegap_mincorr_price_ratio,seat,season_a_s_w,curr_year,start,end,sales_date,supp_region,livm_region,region,phase_in,phase_out,filter,curr_month,Sku_sales_gap_year,prev_year,previous_year_sales,new_sku_flag,economic_impact_risk,quantity_risk,average_gap_risk,average_Sku_sales_gap_year,total_risk,Risk,
                           CASE
                               
                               WHEN Risk == 'Medium' THEN 'Risk_Medium'
                               WHEN Risk ='High' THEN 'Risk_High'
                             ELSE 'Low'
                           END AS Alert,CAI_DESC,
                              brand,
                              DIMBOX,
                              GEOBOX,
                              MFS_IDR,MARKET_CODE,p5_desc,FLAG_ANOMALY
                    FROM data_inter14 da where MFS_IDR= 0 and  brand = 'MICHELIN' and p5_desc IN ('TRUCK RADIAL')  """)

data_inter18.registerTempTable("data_inter18")


In [29]:
final1 = sql("select a.* ,concat(cd.ETRMA_BOUD_CODE,'/',cd.ETRMA_RHS_CODE,' ',cd.ETRMA_STRU_CODE,cd.ETRMA_SEAT_DIA_CODE,' ',cd.ETRMA_CH_I_CODE,cd.ETRMA_VIT_CODE) as DIMBOX1,                                                       concat(cd.ETRMA_BOUD_CODE,'/',cd.ETRMA_RHS_CODE,' ',cd.ETRMA_STRU_CODE,cd.ETRMA_SEAT_DIA_CODE) as GEOBOX1 from data_inter18 a LEFT JOIN cdi_desc cd on cast(a.cai as int) = cast(cd.cai_code as int) ")
final1.registerTempTable("final1")


fianl = sql("""SELECT distinct   country,month,customer,Cust_level3_desc,cai,quantity,sell_in_net_sales,sell_in_net_sales_per_unit,min_corridor_price,max_corridor_price,average_gap_vs_corridor,quantity_in_anomaly,sell_in_loc,economic_impact,quantity_in_anomaly_checked,sell_in_net_sales_in_anomaly,economic_impact_checked,averagegap_mincorr_price_ratio,seat,season_a_s_w,curr_year,start,end,sales_date,supp_region,livm_region,region,phase_in,phase_out,filter,curr_month,Sku_sales_gap_year,prev_year,previous_year_sales,new_sku_flag,economic_impact_risk,quantity_risk,average_gap_risk,average_Sku_sales_gap_year,total_risk,Risk,Alert,CAI_DESC,brand,DIMBOX1 AS DIMBOX,GEOBOX1 AS GEOBOX,MFS_IDR,MARKET_CODE,p5_desc,FLAG_ANOMALY FROM final1""")
fianl.registerTempTable("fianl")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o207.sql.
: org.apache.spark.sql.AnalysisException: Table or view not found: cdi_desc; line 1 pos 350
	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:47)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveRelations$$lookupTableFromCatalog(Analyzer.scala:749)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$.resolveRelation(Analyzer.scala:694)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$$anonfun$apply$8.applyOrElse(Analyzer.scala:730)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$$anonfun$apply$8.applyOrElse(Analyzer.scala:723)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$apply$1.apply(AnalysisHelper.scala:90)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$apply$1.apply(AnalysisHelper.scala:90)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:89)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsUp(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsUp(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$1.apply(AnalysisHelper.scala:87)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$1.apply(AnalysisHelper.scala:87)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:87)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsUp(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsUp(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$1.apply(AnalysisHelper.scala:87)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$1.apply(AnalysisHelper.scala:87)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(Analys

In [30]:
%python
%paste 

TempFilePath = "/mnt/adls/40_project/AAF/Alerting_Flow_TLD/TMP/Parallel_flow_Alert_b2b_compare_tld"
data_inter18.coalesce(1).write\
       .mode("overwrite")\
       .format("com.databricks.spark.csv")\
       .option("header", "true")\
       .save(TempFilePath)\
######## now read file from temp location write it to new location with new name and delete temp directory
readPath = "/mnt/adls/40_project/AAF/Alerting_Flow_TLD/TMP/Parallel_flow_Alert_b2b_compare_tld"
writePath ="/mnt/adls/40_project/AAF/Alerting_Flow_TLD/Output"
file_list = dbutils.fs.ls(readPath) #### List out all files in temp directory
for i in file_list:
  file_path = i[0]
  file_name = i[1]
 #file_name
fname = "Parallel_flow_Alert_b2b_compare_tld.csv"
for i in file_list:
  if i[1].startswith("part-00000"):read_name = i[1] #### find your temp file name
# # # #####Move it outside to the new_dir folder and rename
dbutils.fs.mv(readPath+"/"+read_name, writePath+"/"+fname)
# # # #Remove the empty folder
dbutils.fs.rm(readPath , recurse= True) 

ERROR:root:Getting text from the clipboard on this platform requires tkinter.
Out[31]: True

In [31]:
parelle = spark.read.format("csv").options(header='true', delimiter = ',').load("adl://cdlprdadlsweu.azuredatalakestore.net/40_project/MAD/AGB/ParellelFlow_TLD/Output/Parallel_flow_Alert_b2b_compare_tld.csv")
parelle.registerTempTable("parelle")


--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o276.load.
: java.lang.IllegalArgumentException: No value for fs.adl.oauth2.access.token.provider found in conf file.
	at com.databricks.adl.AdlFileSystem.getNonEmptyVal(AdlFileSystem.java:815)
	at com.databricks.adl.AdlFileSystem.getCustomAccessTokenProvider(AdlFileSystem.java:215)
	at com.databricks.adl.AdlFileSystem.getAccessTokenProvider(AdlFileSystem.java:252)
	at com.databricks.adl.AdlFileSystem.initialize(AdlFileSystem.java:159)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2669)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:370)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:599)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:597)
	at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
	at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike$class.flatMap(TraversableLike.scala:241)
	at scala.collection.immutable.List.flatMap(List.scala:355)
	at org.apache.spark.sql.execution.datasources.DataSource.org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary(DataSource.scala:597)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:391)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:311)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:297)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:214)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 IllegalArgumentException Traceback (most recent call last)
 <command-195397896629628> in <module> 
 ----> 1 parelle = spark . read . format ( "csv" ) . options ( header = 'true' , delimiter = ',' ) . load ( "adl://cdlprdadlsweu.azuredatalakestore.net/40_project/MAD/AGB/ParellelFlow_TLD/Output/Parallel_flow_Alert_b2b_compare_tld.csv" ) 
 2 parelle . registerTempTable ( "parelle" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 164 self . options ( ** options ) 
 165 if isinstance ( path , basestring ) : 
 --> 166 return self . _df ( self . _jreader . load ( path ) ) 
 167 elif path is not None : 
 168 if type ( path ) != list : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *a